In [80]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

file_path = r"C:\Users\Anzer\Desktop\dim_user_features.csv"


In [81]:
df = pd.read_csv(file_path)
df.columns
df = df.fillna(0)

In [82]:
df_inactive = df[df['total_transactions'] == 0]
df_active = df[df['total_transactions'] > 0]

df_inactive.shape
df_active.shape

(50705, 31)

In [96]:
### Defining feature columns

feature_inactive_cols = [
    # demographics
    "age",
    "gender_male",
    "device_android",

    # geography
    "location_urban",
    "location_rural",
    "location_tail",

    # transactions
    #"recency_days",
    #"total_transactions",
    #"median_trx_amount",
    #"credit_card_usage",

    # promo behavior
    #"promo_used_count",
    #"promo_buying_period_ratio",

    # product mix
    #"multi_quantity_ratio",
    #"male_product_ratio",
    #"female_product_ratio",
    #"unisex_product_ratio",
    #"free_items_ratio",
    #"accessories_product_ratio",
    #"footwear_product_ratio",
    #"sporting_goods_product_ratio",
    #"apparel_product_ratio",
    #"home_product_ratio",
    #"personal_care_product_ratio",

    # behavior
    "avg_events_per_booking",
    #"avg_first_event_booking_to_sec",
    "avg_hibrnate_events",
    "avg_search_events",
    "avg_scroll_events",
    "avg_promo_page_events",

    # logistics
    #"ship_hours_diff"
]


### Defining feature columns

feature_active_cols = [
    # demographics
  #  "age",
  #  "gender_male",
  #  "device_android",

    # geography
    "location_urban",
    "location_rural",
  #  "location_tail",

    # transactions
    "recency_days",
    "total_transactions",
    "median_trx_amount",
  #  "credit_card_usage",

    # promo behavior
  #  "promo_used_count",
  #  "promo_buying_period_ratio",

    # product mix
  #  "multi_quantity_ratio",
  #  "male_product_ratio",
  #  "female_product_ratio",
#    "unisex_product_ratio",
 #   "free_items_ratio",
  #  "accessories_product_ratio",
   # "footwear_product_ratio",
    #"sporting_goods_product_ratio",
   # "apparel_product_ratio",
   # "home_product_ratio",
   # "personal_care_product_ratio",

    # behavior
    "avg_events_per_booking",
   # "avg_first_event_booking_to_sec",
   # "avg_hibrnate_events",
   # "avg_search_events",
   # "avg_scroll_events",
   # "avg_promo_page_events",

    # logistics
    #"ship_hours_diff"
]

In [97]:
X_inactive = df_inactive[feature_inactive_cols]
X_active = df_active[feature_active_cols]


X_inactive.shape
X_active.shape

(50705, 6)

In [98]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_active)
X_scaled = pd.DataFrame(X_scaled, columns=X_active.columns)
type(X_scaled)
X_scaled.describe()


,location_urban,location_rural,recency_days,total_transactions,median_trx_amount,avg_events_per_booking
count,5.070500e+04,5.070500e+04,5.070500e+04,5.070500e+04,5.070500e+04,5.070500e+04
mean,1.357886e-16,7.511111e-17,-2.970813e-17,-1.562479e-17,5.072803e-17,4.414179e-17
std,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00
min,-1.235886e+00,-4.736501e-01,-6.856834e-01,-5.322852e-01,-8.660146e-01,-1.644385e+00
25%,-1.235886e+00,-4.736501e-01,-5.962714e-01,-4.986274e-01,-3.264415e-01,-5.094843e-01
50%,8.091359e-01,-4.736501e-01,-4.334956e-01,-3.639960e-01,-2.133506e-01,-1.311841e-01
75%,8.091359e-01,-4.736501e-01,1.373658e-01,3.989821e-02,-4.904164e-02,3.606061e-01
max,8.091359e-01,2.111263e+00,4.406217e+00,1.794587e+01,3.162712e+01,5.636165e+01


In [86]:

# sil_score = {}

# for k in range(2,7):
#     kmeans = KMeans(n_clusters = k, init = 'k-means++', random_state =42)
#     labels = kmeans.fit_predict(X_scaled)
#     sil_score[k] = silhouette_score(X_scaled, labels)

# sil_score



In [102]:
labels = KMeans(5, random_state=42, init='k-means++').fit_predict(X_scaled)
pd.Series(labels).value_counts(normalize=True)


3    0.498531
0    0.181915
4    0.170733
1    0.098965
2    0.049857
Name: proportion, dtype: float64

In [103]:
labels_base = KMeans(4, random_state=42).fit_predict(X_scaled)

from sklearn.metrics import adjusted_rand_score

scores = {}

for col in X_scaled.columns:
    X_temp = X_scaled.drop(columns=[col])
    Xs = StandardScaler().fit_transform(X_temp)
    labels = KMeans(6, random_state=42).fit_predict(Xs)
    scores[col] = adjusted_rand_score(labels_base, labels)

sorted(scores.items(), key=lambda x: x[1])


[('location_urban', 0.35767771130346426),
 ('recency_days', 0.6903991237045057),
 ('total_transactions', 0.7346217825754355),
 ('median_trx_amount', 0.7989124598027336),
 ('avg_events_per_booking', 0.8149515605224811),
 ('location_rural', 0.924427845855931)]

In [105]:
import pandas as pd
import numpy as np

corr = pd.DataFrame(X_scaled).corr().abs()

upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
drop_cols = [c for c in upper.columns if any(upper[c] > 0.85)]

drop_cols


[]

In [106]:
drop_cols

[]